In [1]:
import os
import numpy as np
import pandas as pd
from glob import glob
import tensorflow as tf
from tensorflow.keras import layers, models, utils, callbacks, Input, Model
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter

In [ ]:
def load_npz_data(data_dir, prefix):
    Xfile = np.load(f"{data_dir}/X_{prefix}_beats.npz")
    Yfile = np.load(f"{data_dir}/y_{prefix}_beats.npz")
    # Ambil array pertama (apapun key-nya)
    X = Xfile[Xfile.files[0]]
    Y = Yfile[Yfile.files[0]]
    return X, Y

In [11]:
data_dir = 'data_ready_nonsegment'
X_train, y_train = load_npz_data(data_dir, 'train')
X_val, y_val = load_npz_data(data_dir, 'valid')
X_test, y_test = load_npz_data(data_dir, 'test')

print("Train beats :", X_train.shape, y_train.shape)
print("Valid beats :", X_val.shape, y_val.shape)
print("Test beats  :", X_test.shape, y_test.shape)

Train beats : (15286, 1000, 12) (15286, 5)
Valid beats : (4367, 1000, 12) (4367, 5)
Test beats  : (2184, 1000, 12) (2184, 5)


In [ ]:
from scipy.signal import butter, filtfilt

def butter_lowpass_filter_ecg(X, cutoff=40, fs=100, order=2):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    
    # X.shape = (samples, time_steps, channels)
    X_filtered = filtfilt(b, a, X, axis=1)  # filter sepanjang axis waktu (1000)
    return X_filtered

X_train_filtered = butter_lowpass_filter_ecg(X_train, cutoff=40, fs=100)
X_val_filtered   = butter_lowpass_filter_ecg(X_val, cutoff=40, fs=100)
X_test_filtered  = butter_lowpass_filter_ecg(X_test, cutoff=40, fs=100)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import plot_model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt

# Arsitektur stacked LSTM
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(1000, 12)),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(5, activation='softmax')  # output untuk 5 kelas
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Melatih model
history = model.fit(
    X_train_filtered, y_train,
    validation_data=(X_val_filtered, y_val),
    epochs=20,
    batch_size=128
)

# Evaluasi
loss, acc = model.evaluate(X_test, X_test_filtered)
print(f"Test Loss : {loss:.4f}")
print(f"Test Accuracy : {acc:.4f}")


Epoch 1/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 216s 2s/step - accuracy: 0.3841 - loss: 2.0579 - val_accuracy: 0.4573 - val_loss: 2.0033
Epoch 2/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 224s 2s/step - accuracy: 0.3718 - loss: 2.1064 - val_accuracy: 0.4578 - val_loss: 2.0026
Epoch 3/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 229s 2s/step - accuracy: 0.3698 - loss: 2.0958 - val_accuracy: 0.4578 - val_loss: 1.9936
Epoch 4/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 232s 2s/step - accuracy: 0.3549 - loss: 2.0959 - val_accuracy: 0.4578 - val_loss: 1.9780
Epoch 5/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 236s 2s/step - accuracy: 0.3533 - loss: 2.1055 - val_accuracy: 0.4578 - val_loss: 2.0118
Epoch 6/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 243s 2s/step - accuracy: 0.3410 - loss: 2.1121 - val_accuracy: 0.4578 - val_loss: 2.0273
Epoch 7/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 252s 2s/step - accuracy: 0.3536 - loss: 2.1373 - val_accuracy: 0.4578 - val_loss: 2.0831
Epoch 8/20
120/120 ━━━━━━━━━━━━━━━━━━━━ 256s 2s/step - accuracy: 0.3351 - loss: 2.1809 - val_accu

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# =============================
# 7. Confusion Matrix
# =============================
y_pred_prob = model.predict(X_test_filtered)
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_true, y_pred)
labels = [f'Class {i}' for i in range(5)]

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
fig, ax = plt.subplots(figsize=(7, 7))
disp.plot(ax=ax, cmap='Blues', values_format='d')
plt.title("Confusion Matrix")
plt.show()

print("Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=labels))